# Generalizations of Max-Flow

The purpose of this assignment is to learn about the min-cost flow problem, a generalization of max-flow, and to familiarize yourself with implementing and solving linear programs.

## Min-Cost Flow

Recall that a flow network with demands consists of a directed graph $G = (V, E)$, where each edge $(i,j) \in E$ has a positive integer capacity $c_{ij}$ and each node $i \in V$ has an integer demand $d_i$. In a *min-cost flow* problem, each edge $(i,j) \in E$ also has a cost (or weight) $w_{ij}$. (Note that this input generalizes the input to two important problems we discussed so far: in max flow, the edge weights were not important while in shortest paths, the edge capacities were not important.) 

Given a flow network with capacities and costs, the goal is to find a *feasible* flow $f: E \rightarrow R^+$ --that is, a flow satisfying edge capacity constraints and node demands-- that minimizes the total cost of the flow. Explicitly, the problem can be formulated as a linear program.

### Question 1

Answer Problem 1 in HW4-theoretical.

### Question 2

To implement your reduction from Problem 1 in HW4-theoretical, you will work with some standard benchmark instances for min-cost flow found [here](http://elib.zib.de/pub/Packages/mp-testdata/mincost/gte/index.html). The format of the data is described in the [Info](http://elib.zib.de/pub/Packages/mp-testdata/mincost/gte/info) file. You are to read in the graph from the data file in a form that can be solved using NetworkX's `min_cost_flow` function. Note that the data sometimes lists multiple edges between the same nodes, but with different costs or capacities. In forming the graph, you need to implement your reduction from the previous question and form a `DiGraph` instance, because the `min_cost_flow` function cannot handle multi-edges, even though the package offers `MultiDiGraph` objects.

In [1]:
import networkx as nx

def create_graph(infile):
    
    """Creates a directed graph as specified by the input file. Edges are annotated with 'capacity'
    and 'weight' attributes, and nodes are annotated with 'demand' attributes.
    
    Args:
        infile: the input file using the format to specify a min-cost flow problem.
        
    Returns:
        A directed graph (but not a multi-graph) with edges annotated with 'capacity' and 'weight' attributes
        and nodes annotated with 'demand' attributes.
    """
    # TODO: implement function   
    G=nx.DiGraph()
    graph = open(infile)
    counter=0
    for line in graph:
        
        # extract all nodes with demands
        if line[0]=='n':
            n,n_id,n_demand=line.strip().split()
            G.add_node(n_id)
            G.node[n_id]['demand']=int(n_demand)
            
        # extract all edges (arcs) with their properties viz. Upper bound Capacity,Lower Bound Capacity, Weights
        if line[0]=='a':
            a1,a2,a3,a4,a5,a6=line.strip().split()
            
            # Reduction of graph when it encounters multiple edges between two nodes
            if G.has_edge(a2,a3):
                x='x'+str(counter)
                
                G.add_edge(a2,x)
                G.edge[a2][x]['capacity']=int(a5)
                G.edge[a2][x]['weight']=0
                G.edge[a2][x]['lower']=int(a4)
            
                G.add_edge(x,a3)
                G.edge[x][a3]['capacity']=int(a5)
                G.edge[x][a3]['weight']=int(a6)
                G.edge[x][a3]['lower']=int(a4)
                counter=counter+1
            else:  
                G.add_edge(a2,a3)
                G.edge[a2][a3]['capacity']=int(a5)
                G.edge[a2][a3]['weight']=int(a6)
                G.edge[a2][a3]['lower']=int(a4)
    graph.close()
    
    # Assigning demand of all other nodes = 0
    for i in G.nodes():
        if G.node[i]=={}:
            G.node[i]['demand']=0
        else:
            continue
    return G


    pass

The following will check that your code outputs the expected min cost flow values on several test instances.

In [2]:
G_40 = create_graph('gte_bad.40')
G_6830 = create_graph('gte_bad.6830')
G_176280 = create_graph('gte_bad.176280')

print "Correct value for _40 instance:", nx.min_cost_flow_cost(G_40) == 52099553858
print "Correct value for _6830 instance:", nx.min_cost_flow_cost(G_6830) == 299390431788
print "Correct value for _176280 instance:", nx.min_cost_flow_cost(G_176280) == 510585093810

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True


## Linear Programming

Instead of using special-purpose min-cost flow solvers, you will now formulate the problems as linear programs and use general-purpose LP solvers to find the solutions.

### Question 3

Implement the following function to formulate the flow LP and return the optimal value (i.e., minimum cost over feasible flows).

In [3]:
import pulp

def lp_flow_value(G):
    """Computes the value of the minimum cost flow by formulating and solving
    the problem as an LP.
    
    Args:
        G: a directed graph with edges annotated with 'capacity' and 'weight'
            attrbutes, and nodes annotated with 'demand' attributes.
            
    Returns:
        The value of the minimum cost flow.
    """
    # TODO: implement function
    flow={}
    counter=1
    obj=[]
    
    # define flow dictionary for each node with inflow, outflow and demand
    for i in G.nodes():
        flow[i]={'inflow':[], 'outflow':[], 'demand':G.node[i]['demand']}
                 
    for (s1,s2) in G.edges() :
        f=pulp.LpVariable('f'+str(counter),G.edge[s1][s2]['lower'], G.edge[s1][s2]['capacity'])
        flow[s1]['outflow'].append(f)
        flow[s2]['inflow'].append(f)
        obj.append(f*G.edge[s1][s2]['weight'])
        counter=counter+1
        
    # Objective to minimize cost
    prob = pulp.LpProblem("problem", pulp.LpMinimize)
    
    # Demand contraints for each node defined
    for i in G.nodes():     
        prob += sum(flow[i]['inflow'])-sum(flow[i]['outflow']) <= flow[i]['demand']
    
    # Solve LP
    prob += sum(obj)
    status = prob.solve()
    return pulp.value(sum(obj))
    pass

The following will check that the LP finds the same optimal values as previously.

In [4]:
print "Correct value for _40 instance:", lp_flow_value(G_40) == 52099553858
print "Correct value for _6830 instance:", lp_flow_value(G_6830) == 299390431788
print "Correct value for _176280 instance:", lp_flow_value(G_176280) == 510585093810

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True
